# Construindo um Pipeline ETL com Python - Santander Dev Week 2023

Este é um desafio originario do Santander Bootcamp 2023 - Ciência de Dados com Python.


## Bibliotecas Necessárias 📚

Nesta seção, instalaremos e importaremos as bibliotecas necessárias para o projeto, incluindo `pandas`, `requests`, e `google-generativeai`. Certifique-se de ter essas bibliotecas instaladas ou utilize a célula abaixo para instalação.


In [1]:
!pip install google-generativeai
import pandas as pd
import requests
import json
import google.generativeai as genai
import os

c:\Users\boeni\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configurações Iniciais

Para utilizar a API, defina a URL fornecida pela organização do evento ou configure a sua própria instância da API conforme necessário.

Essa URL será usada para extrair os dados na próxima etapa.

In [1]:
# URL da API do Santander Dev Week 2023
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## ETL (Extract, Transform, Load) 🚀

Abaixo, estão as três etapas do processo ETL detalhadas:

### 1. Extract (Extração) ⏬

Nesta etapa, os dados são extraídos da API do evento. A função de extração utiliza `requests` para fazer uma chamada GET à API e retorna os dados em formato JSON.


In [3]:
# Etapa 1: Capturar os IDS da lista CSV
df = pd.read_csv("SDW2023.csv")
user_ids = df["UserID"].tolist()
print(user_ids)

[4573, 5542, 3441]


In [4]:
# Etapa 2: Capturar dados dos usuários com a API da SDW
def get_user(id):
  response = requests.get(f"{sdw2023_api_url}/users/{id}")
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 4573,
    "name": "Isa",
    "account": {
      "id": 4850,
      "number": "354119444",
      "agency": "string",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 4437,
      "number": "121162255",
      "limit": 0.0
    },
    "features": [],
    "news": [
      {
        "id": 8599,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "A import\u00e2ncia dos investimentos para Isa: Garantir um futuro financeiro seguro e pr\u00f3spero."
      },
      {
        "id": 8940,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": ""
      },
      {
        "id": 8941,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Isa, fa\u00e7a seu dinheiro trabalhar para voc\u00ea. Invista com confian\u00e7a! #CrescimentoFinanceiro"
      }

### 2. Transform (Transformação) 🔃

Após a extração dos dados, realizamos a transformação necessária. Aqui, os dados podem ser filtrados, manipulados e estruturados de acordo com os requisitos do projeto.

In [5]:
#Etapa 1: Adição da sua KEY única da Gemini IA ou de sua preferencia.
gemini_api_key = "SUA_API_KEY_GEMINI"
genai.configure(api_key=gemini_api_key)

In [7]:
#Etapa 2: Geração de frases utilizando a Gemini IA (código pode variar de acordo com a IA que preferir)

def generate_ia_news(user):
  try:
      model = genai.GenerativeModel('gemini-1.5-pro')
      prompt = f"Surpreenda o cliente criando uma mensagem personalizada e impactante para {user['name']} sobre a importância da consciencia financeira (máximo de 100 caracteres)(no maximo 2 emojis)"
      response = model.generate_content(prompt)
      return response.text.strip()
  except Exception as e:
      print(f"Erro ao gerar mensagem: {e}")
      return None

for user in users:
  news = generate_ia_news(user)
  print(news)

Isa,💰 realizar seus sonhos começa com cuidar do seu presente. 😉 Comece a trilhar sua liberdade financeira!
Valdemort,💰 futuro tranquilo é planejado hoje! 😉  Comece a trilhar o seu caminho para a liberdade financeira.
Pepe, grana na mão é liberdade na vida! 💰 Aprenda a cuidar bem dela e conquiste seus sonhos. 😉


### 3. Load (Carregamento) 🔄️

Nesta última etapa, os dados processados são carregados para um destino final, que pode ser um banco de dados, arquivo ou qualquer outro repositório de armazenamento.

In [8]:
#Atualizar os dados do Santander Dev Week com os dados enriquecidos
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Isa updated? True!
User Valdemort updated? True!
User Pepe updated? True!


## Conclusão ✅

Este notebook apresentou o processo ETL aplicado aos dados da API do Santander Dev Week 2023. As etapas incluíram a extração de dados de uma API, a transformação para adequação ao formato desejado e o carregamento para o destino final.
